# GRPO 

[deepseek-R1](https://github.com/deepseek-ai/DeepSeek-R1)

[HF:GRPO Trainer](https://huggingface.co/docs/trl/grpo_trainer)

[Approximating KL Divergence](http://joschu.net/blog/kl-approx.html)

[Deepseek math](https://arxiv.org/abs/2402.03300)

## Why GRPO
1. PPO $\rightarrow$ DPO $\rightarrow$ GRPO 
    - ppo 需要单独训练一个reward model 进行训练，架构复杂且算力需求量大
      - 目标：
        $$
        \max_{\theta} \mathbb{E}_{(x,y) \sim \pi_{\theta}} \left[ r_{\phi}(x,y) - \beta \, \text{KL}(\pi_{\theta} \| \pi_{\text{ref}}) \right]
        $$
    - DPO 在 PPO 的基础上改进了 reward model 写出其显示解，使用偏好数据对(chosen, rejucted)：
      - $$
        r(x, y) \approx \beta \left( \log \pi_{\theta}(y|x) - \log \pi_{\text{ref}}(y|x) \right)
        $$
        - 这样就显式地写出损失函数：
        $$
        \mathcal{L}_{\text{DPO}} = -\mathbb{E}_{(x, y^+, y^-)} \left[ \log \sigma \left( \beta (\Delta_{\theta} - \Delta_{\text{ref}}) \right) \right]
        $$  
         - 其中 $\Delta_{\theta} = \log \pi_{\theta}(y^+|x) - \log \pi_{\theta}(y^-|x)$。
    - GRPO ：在复杂的任务，一个 prompt 通常会有多个候选答案，不只是但一个评判标准--好/坏
      - 主要改进在reward：
        - 把多个候选作为一组（group），组内进行比较。
        - 同样不训练 critic ，不需要显示的reward model，使用**组内的相对评分**用来估计优势函数

        

## GRPO具体流程
对于每一个question q，GRPO从旧的策略 $\pi_{\theta_{old}}$ 采样一组输出 ${o}_i^{G} = \{o_1, o_2, ..., o_G \} $，并且通过最大化下面的目标：
$$
J_{G R P O}(\theta)=\mathbb{E}[q \sim P(Q),\{o_{i}\}_{i=1}^{G} \sim \pi_{\theta_{o l d}}(O|q)]
$$
$$
\frac{1}{G} \sum_{i=1}^{G} \frac{1}{|o_{i}|} \sum_{t=1}^{|o_{i}|}\left\{\min \left[\frac{\pi_{\theta}(o_{i, t}|q, o_{i,<t})}{\pi_{\theta_{o l d}}(o_{i, t}|q, o_{i,<t})} \hat{A}_{i, t}, \operatorname{clip}\left(\frac{\pi_{\theta}(o_{i, t}|q, o_{i,<t})}{\pi_{\theta_{o l d}}(o_{i, t}|q, o_{i,<t})}, 1-\epsilon, 1+\epsilon\right) \hat{A}_{i, t}\right]-\beta \mathrm{D}_{\mathrm{KL}}\left[\pi_{\theta} \| \pi_{\mathrm{ref}}\right]\right\},
$$

![PPO and GRPO](./img/PPO-and-GRPO.png)

2. GRPO 流程（下半部分）

GRPO 的改进在于 **去掉** Value Model，用 **组内比较**(Group-based Advantage Estimation) 代替基线。

- **Step 1: 输入** \( q \)
  - Policy Model 根据 prompt \( q \) 生成一组候选输出：
    $$
    o_1, o_2, \ldots, o_G
    $$
- **Step 2: 评估**
  - 每个输出通过 Reference Model 计算 KL 惩罚。
  - 每个输出通过 Reward Model 得到奖励分数：
    $$ r_1, r_2, \ldots, r_G$$

- **Step 3: Group Computation**
  - 不再用 Value Model 来估计 baseline，而是利用 **组内统计量**（如平均分或 pairwise 比较）来估计基线。
  - 对每个输出计算 advantage：
    $$ A_i = r_i - \text{baseline(group)}$$
    - baseline(group) 可以是组内平均奖励，也可以是其他相对评分方式。

- **Step 4: Policy 更新**
  - 使用 PPO 的 clipped objective，但 advantage 来自 **组内相对优势**，而非 value network。

![GRPO method](img/GRPO-method.png)


![algorithm](./img/GRPO-Algorithm.png)

## More detials
### 1.Outcome Supervision RL with GRPO

对于一个输入问题 \( q \)，采样 \( G \) 个输出：
$$
\{ o_1, o_2, \ldots, o_G \}
$$
Reward model 给出对应奖励：
$$
r = \{ r_1, r_2, \ldots, r_G \}
$$
然后做组内归一化（z-score）：
$$
\hat{r}_i = \frac{r_i - \text{mean}(r)}{\text{std}(r)}
$$
这里的 baseline 就是组平均 \(\text{mean}(r)\)，同时再除以 \(\text{std}(r)\) 做标准化，避免 reward scale 不稳定。
最后所有 token 的优势函数：
$$
\hat{A}_{i, t} = \hat{r}_i
$$

### 2.Process Supervision RL with GRPO

区别在于 **奖励不是只有最终答案**，而是对中间每个 reasoning step 都打分。

假设第 \( i \) 个输出有 \( K_i \) 步，每一步的奖励是：

$$
r_i^{\text{index}(j)} \quad (j = 1, 2, \ldots, K_i)
$$

组内 baseline 的计算方式同样是：

$$
\tilde{r}_i^{\text{index}(j)} = \frac{r_i^{\text{index}(j)} - \text{mean}(R)}{\text{std}(R)}
$$

其中 \( R \) 是所有 group 中所有 step 的 reward 集合。

优势函数计算：

$$
\hat{A}_{i, t} = \sum_{\text{index}(j) \geq t} \tilde{r}_i^{\text{index}(j)}
$$

直观理解：

- 每一步推理都会得到一个 reward，避免只在最后一步监督；
- baseline 仍然是组平均 + 标准化，只不过是在 step-level。


### 3.KL散度

$$
\mathbb{D}_{KL} \left[ \pi_{\theta} \| \pi_{\text{ref}} \right] = \frac{\pi_{\text{ref}}(o_{i,t}|q, o_{i,<t})}{\pi_{\theta}(o_{i,t}|q, o_{i,<t})} - \log \frac{\pi_{\text{ref}}(o_{i,t}|q, o_{i,<t})}{\pi_{\theta}(o_{i,t}|q, o_{i,<t})} - 1,
$$
